In [182]:
# Проверка возможности использования xgbooster в основном, либо предварительно (2017-06-30)

In [183]:
import sys,os,datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import fbeta_score
from __future__ import print_function

In [184]:
print(np.__version__);
print(pd.__version__);

1.12.1
0.19.2


In [185]:
import  cv2 as cv
cv.__version__

'3.2.0'

In [186]:
sys.path.append('../Python')
from helper import formFH, paths_input

In [223]:
binsData = 255

trLabels,trDirTIF,trDirJPG,teDirTIF,teDirJPG = paths_input()
if True :
    trDirI = trDirJPG
    teDirI = teDirJPG
    ttExt  = '.jpg'
else :
    trDirI = trDirTIF
    teDirI = teDirTIF
    ttExt  = '.tif'
trWork, teWork = '../Work/Train', '../Work/Test'
print (trDirI,teDirI, trWork, teWork)
print (trLabels,trDirTIF,trDirJPG,teDirTIF,teDirJPG)

../Data/train-jpg ../Data/test-jpg-v2 ../Work/Train ../Work/Test
../Data/train_v2.csv ../Data/train-tif-v2 ../Data/train-jpg ../Data/test-tif-v2 ../Data/test-jpg-v2


In [188]:
xx=os.listdir(trDirI); xx.sort(); xx[:6]

['train_0.jpg',
 'train_1.jpg',
 'train_10.jpg',
 'train_100.jpg',
 'train_1000.jpg',
 'train_10000.jpg']

In [189]:
labels_df = pd.read_csv(trLabels)
labels_df.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [190]:
# Build list with unique labels
label_list = []
for tag_str in labels_df.tags.values:
    labels = tag_str.split(' ')
    for label in labels:
        if label not in label_list:
            label_list.append(label)

In [191]:
# Add features (0 or 1) for every label as dataframe field 
for label in label_list:
    labels_df[label] = labels_df['tags'].apply(lambda x: 1 if label in x.split(' ') else 0)
#labels_df.head()

In [192]:
weather_labels = ['clear', 'partly_cloudy', 'haze', 'cloudy']
land_labels = ['primary', 'agriculture', 'water', 'cultivation', 'habitation' ]
rare_labels = [l for l in label_list if labels_df[label_list].sum()[l] < 2000]
#rare_labels              

In [193]:
labels = label_list; #weather_labels;
nameList =labels_df[labels_df[labels].sum(axis=1)>0].image_name.tolist(); len(nameList)
labelList=labels_df[labels_df[labels].sum(axis=1)>0][labels].as_matrix();
labelList[:6,:]
#labels_df.head()

array([[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [194]:
def formImExt (nf, resize=(32,32), printOK=False) :
    nx = None
    try : 
        ni = cv.imread(nf,-1); 
        if (ni is not None) :
            if not ((ni.shape[2]==3) or (ni.shape[2]==4)) and printOK : print('----- error ---- shape:',ni.shape,nf)
            if (ni.shape[2]==3) :
                nx = cv.resize(ni,resize)
            if (ni.shape[2]==4) :
                #r,g,b,n = ni[:,:,2],ni[:,:,1],ni[:,:,0],ni[:,:,3]
                r,g,b,n = cv.resize(ni[:,:,2],resize),cv.resize(ni[:,:,1],resize),cv.resize(ni[:,:,0],resize),cv.resize(ni[:,:,3],resize)
                dv,dw   = np.divide((r-n),(r+n+0.01)), np.divide((g-n),(g+n+0.01))
                nx      = np.array([r,g,b,n,dv,dw]).T; 
    except BaseException as e :
        print(nf,e); nx = None;
    
    if nx is None and printOK : 
        print('------ None:',nf); nx = None
        
    return(nx)

def formImHist (nf, size, printOK=False) :
    
    def hist1 (nf1,bins) :
        h,_ = np.histogram(nf1.ravel(),bins=bins)
        return h.tolist()
    
    def histN (nf,bins) :
        h = []
        for i in range(0,nf.shape[2]) :
            h = h + hist1(nf[:,:,i],bins=bins);
        return h
             
    nx = None
    try : 
        ni = cv.imread(nf,-1);
        if (ni is not None) :
            
            if printOK : print(nf,ni.shape)

            if not ((ni.shape[2]==3) or (ni.shape[2]==4)) and printOK : 
                print('----- error ---- shape:',ni.shape,nf);
            if  ni.shape[2]==3 :
                bins = [x for x in range(0,255+int(round(256.0/size)),int(round(256.0/size)))]
                nx = histN(ni,bins)
            if  ni.shape[2]==4 : 
                bins = [x for x in range(0,65535+int(round(65535.0/size)),int(round(65535.0/size)))]
                nx = histN(ni,bins)
            
    except BaseException as e :
        print(nf,e); nx = None;
    
    if nx is None and printOK : 
        print('------ None:',nf); nx = None
        
    return(nx)

In [195]:
os.path.exists(nf)
ni = cv.imread(nf,-1);
ni.shape

(256, 256, 3)

In [224]:
trX, trY, i, size = [],[], 0, len(nameList)
print(datetime.datetime.now(),len(nameList),size)
for nn in nameList[0:size] :
    nf = os.path.join(trDirI,nn+".jpg");
    nx = formImHist(nf,binsData,printOK=False)
    if (nx is not None) :
        #rr=np.save(os.path.join(trWork,nn+".npy"),nx);
        #trX.append(nn+".npy")
        trX.append(nx)
        trY.append(True)
    else : 
        trY.append(False)
    i += 1
    if (i%5000==0) : print(datetime.datetime.now(),"\t",i,"\t",nn)
    #print(nn.shape)

print(datetime.datetime.now())
trX = np.array(trX);
trY = labelList[trY];
print(len(nameList),trX.shape,trY.shape)
#trXY=pd.DataFrame(trY); trXY['name']=trX; trXY.head()
#trXY.to_pickle(os.path.join(trWork,"listFiles.pkl"))

2017-07-01 05:20:44.191314 40479 40479
2017-07-01 05:21:17.194838 	 5000 	 train_4999
2017-07-01 05:21:50.173692 	 10000 	 train_9999
2017-07-01 05:22:22.653734 	 15000 	 train_14999
2017-07-01 05:22:54.940745 	 20000 	 train_19999
2017-07-01 05:23:26.515550 	 25000 	 train_24999
2017-07-01 05:23:57.473847 	 30000 	 train_29999
2017-07-01 05:24:30.076561 	 35000 	 train_34999
2017-07-01 05:25:02.784989 	 40000 	 train_39999
2017-07-01 05:25:05.922457
40479 (40479, 765) (40479, 17)


In [225]:
trX.shape,trY.shape

((40479, 765), (40479, 17))

In [226]:
np.save('../Work/train-data-XX.npy',trX)
np.save('../Work/train-data-YY.npy',trY)

In [227]:
del(trX); del(trY);

In [228]:
trX = np.load('../Work/train-data-XX.npy')
trY = np.load('../Work/train-data-YY.npy')
print(trX.shape,trY.shape)

(40479, 765) (40479, 17)


In [229]:
import xgboost as xgb

In [230]:
# Строим основные массивы
dtrain = xgb.DMatrix(trX,label=trY[:,1])

In [231]:
param = {
         'objective':'reg:logistic', 
         #'num_class':17,
         #'eta':0.05, 
         #'max_depth':9, 'subsample':0.8, 'colsample_bytree':0.8, 
         #'metrics':'rmse'
         'eval_metric':['auc','rmse']
        }
param['silent']=True

results, bstList = [], []
for i in range(0,trY.shape[1]) :
    print ('Working: {}'.format(i))
    dtrain.set_label(trY[:,i])
    history = xgb.cv(param, dtrain, metrics=['auc'], verbose_eval=30, num_boost_round=1000, nfold=3, early_stopping_rounds=20) #,stratified=True,maximize=True)
    results.append(history)
    rvalidCV = {}
    watchlistCV  = [(dtrain,'train')]
    bst = xgb.train(param, dtrain, history['test-auc-mean'].argmax()+1, watchlistCV) 
    bstList.append(bst)
    

Working: 0
[0]	train-auc:0.943547+0.00524291	test-auc:0.929328+0.00505594
[30]	train-auc:0.988459+0.000516729	test-auc:0.958506+0.00232698
[60]	train-auc:0.996101+0.000172513	test-auc:0.958487+0.0025079
[0]	train-auc:0.943299	train-rmse:0.377534
[1]	train-auc:0.948933	train-rmse:0.300219
[2]	train-auc:0.958053	train-rmse:0.250566
[3]	train-auc:0.9606	train-rmse:0.220209
[4]	train-auc:0.963582	train-rmse:0.201258
[5]	train-auc:0.966638	train-rmse:0.189529
[6]	train-auc:0.968566	train-rmse:0.182326
[7]	train-auc:0.969726	train-rmse:0.177749
[8]	train-auc:0.970854	train-rmse:0.175135
[9]	train-auc:0.972179	train-rmse:0.172977
[10]	train-auc:0.973048	train-rmse:0.171026
[11]	train-auc:0.974115	train-rmse:0.170041
[12]	train-auc:0.97567	train-rmse:0.168424
[13]	train-auc:0.976395	train-rmse:0.167968
[14]	train-auc:0.977073	train-rmse:0.167384
[15]	train-auc:0.977724	train-rmse:0.16678
[16]	train-auc:0.978741	train-rmse:0.165077
[17]	train-auc:0.979507	train-rmse:0.163399
[18]	train-auc:0.97

In [112]:
#results[0]

In [113]:
#[(x['test-rmse-mean'].min(),x['test-rmse-mean'].argmin()) for x in results]

In [232]:
#[(x['test-rmse-mean'].min(),x['test-rmse-mean'].argmin()) for x in results]

In [161]:
[(x['test-auc-mean'].max(),x['test-auc-mean'].argmax()) for x in results] # 50

[(0.95933766666666676, 51),
 (0.97132033333333334, 70),
 (0.92636800000000008, 84),
 (0.97527066666666673, 52),
 (0.82370966666666667, 91),
 (0.93564866666666668, 41),
 (0.91020533333333331, 58),
 (0.84959400000000007, 84),
 (0.84758433333333338, 38),
 (0.98782933333333334, 197),
 (0.9787530000000001, 214),
 (0.95955733333333326, 49),
 (0.86465266666666662, 54),
 (0.98490033333333338, 55),
 (0.88952866666666663, 32),
 (0.87565166666666672, 27),
 (0.81941499999999989, 40)]

In [208]:
[(x['test-auc-mean'].max(),x['test-auc-mean'].argmax()) for x in results] # 75

[(0.95963833333333337, 30),
 (0.97144833333333336, 73),
 (0.92635233333333333, 109),
 (0.97506933333333334, 130),
 (0.82618333333333327, 151),
 (0.93428500000000003, 56),
 (0.91021033333333345, 56),
 (0.84739166666666665, 74),
 (0.84609366666666663, 31),
 (0.98742033333333323, 82),
 (0.97728066666666669, 190),
 (0.957928, 27),
 (0.86184966666666674, 40),
 (0.98402533333333331, 56),
 (0.88504333333333329, 28),
 (0.87253733333333328, 39),
 (0.83189766666666676, 36)]

In [233]:
[(x['test-auc-mean'].max(),x['test-auc-mean'].argmax()) for x in results] # 255

[(0.95898666666666665, 42),
 (0.97181966666666675, 152),
 (0.9253096666666667, 73),
 (0.97478400000000009, 114),
 (0.82098633333333326, 109),
 (0.92782300000000006, 36),
 (0.9083026666666667, 54),
 (0.8458106666666666, 37),
 (0.85382900000000006, 57),
 (0.98749666666666658, 226),
 (0.97769933333333336, 158),
 (0.95703633333333338, 26),
 (0.86158199999999996, 33),
 (0.98312966666666668, 57),
 (0.88108266666666657, 39),
 (0.87651833333333329, 58),
 (0.8118953333333333, 43)]

In [255]:
# Forming output dataset for predicting --> trOX, trOY
del(trX)
del(trY)

In [234]:
nameAsk = os.listdir(teDirI); print(len(nameAsk))
trOX, trOY, i, size = [], [], 0, len(nameAsk)
print(datetime.datetime.now(),len(nameAsk),size)
for nn in nameAsk[0:size] :
    nf = os.path.join(teDirI,nn);
    nx = formImHist(nf,binsData) #,printOK=True)
    if (nx is not None) :
        trOX.append(nx)
        trOY.append(nn)
    i += 1
    if (i%5000==0) : print(datetime.datetime.now(),"\t",i,"\t",nn)

print(datetime.datetime.now())

#assert (size!=len(trOY)), "Wrong files {} != {}".format(size,len(trOY))

61191
2017-07-01 05:47:35.169662 61191 61191
2017-07-01 05:48:07.160675 	 5000 	 test_14523.jpg
2017-07-01 05:48:39.319640 	 10000 	 test_19029.jpg
2017-07-01 05:49:11.122597 	 15000 	 test_23524.jpg
2017-07-01 05:49:43.459654 	 20000 	 test_28015.jpg
2017-07-01 05:50:15.849019 	 25000 	 test_32520.jpg
2017-07-01 05:50:48.009813 	 30000 	 test_37026.jpg
2017-07-01 05:51:20.076443 	 35000 	 test_4908.jpg
2017-07-01 05:51:51.023611 	 40000 	 test_9402.jpg
2017-07-01 05:52:22.607948 	 45000 	 file_13913.jpg
2017-07-01 05:52:55.296418 	 50000 	 file_18419.jpg
2017-07-01 05:53:27.779680 	 55000 	 file_4564.jpg
2017-07-01 05:54:00.158926 	 60000 	 file_892.jpg
2017-07-01 05:54:07.893954


In [235]:
trOX = np.array(trOX);
trOY = np.array([os.path.splitext(x)[0] for x in trOY]);
print(len(nameAsk),trOX.shape,trOY.shape)

61191 (61191, 765) (61191,)


In [236]:
#trOX[20],trOY[20]

In [237]:
# Saving & Loading

In [238]:
np.save('../Work/test-data-XX.npy',trOX)
np.save('../Work/test-data-YY.npy',trOY)

In [239]:
trOX = np.load('../Work/test-data-XX.npy')
trOY = np.load('../Work/test-data-YY.npy')
print(trOX.shape,trOY.shape)

(61191, 765) (61191,)


In [240]:
# Формирование результата
dtest  = xgb.DMatrix(trOX)

In [241]:
trP = []
for bstI in bstList :
    trP1 = bstI.predict(dtest);
    trP.append(trP1)
trP = np.array(trP).T
type(trP),trP.shape    

(numpy.ndarray, (61191, 17))

In [242]:
np.save('../Work/test-data-RR.npy',trP)

In [243]:
trP = np.load('../Work/test-data-RR.npy')
trP.shape

(61191, 17)

In [244]:
wr = [labels.index(i) for i in weather_labels];
trM=np.array([0.5]*17);
trM[np.array(wr)] = 2.0
print(wr,'\n',labels,'\n',trM.tolist())

[3, 10, 0, 9] 
 ['haze', 'primary', 'agriculture', 'clear', 'water', 'habitation', 'road', 'cultivation', 'slash_burn', 'cloudy', 'partly_cloudy', 'conventional_mine', 'bare_ground', 'artisinal_mine', 'blooming', 'selective_logging', 'blow_down'] 
 [2.0, 0.5, 0.5, 2.0, 0.5, 0.5, 0.5, 0.5, 0.5, 2.0, 2.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]


In [245]:
#trP = model.predict(trX, batch_size=512); trP=K.get_value(trP)
res = []

for i in range(trP.shape[0]) :
    trA1 = [weather_labels[trP[i,wr].argmax()]]
    trA2 = [labels[ii] for ii in range(len(labels)) if (trP[i,ii]>trM[ii])]
    trPP = trA1 + trA2;
    pp   = ' '.join(trPP)
    ##if (pp=="") : print(trY[i])
    res.append([trOY[i],pp])

res.sort(cmp=lambda x,y: cmp(int(x[0].partition('_')[2]),int(y[0].partition('_')[2])) if (x[0].partition('_')[0]==y[0].partition('_')[0]) else cmp(y[0].partition('_')[0],x[0].partition('_')[0]))
#print(res[4:8],'\n',res[-4:])
print(res[0],'\n',res[-1],'\n',trM,trP[0])

['test_0', 'clear primary'] 
 ['file_20521', 'clear primary'] 
 [ 2.   0.5  0.5  2.   0.5  0.5  0.5  0.5  0.5  2.   2.   0.5  0.5  0.5  0.5
  0.5  0.5] [  1.36969104e-01   1.69111237e-01   4.36139032e-02   1.97642162e-01
   4.44384873e-01   4.44027176e-03   9.07832570e-03   5.52273169e-03
   2.23991792e-05   9.40030217e-01   7.28166953e-04   2.10852086e-04
   4.28348500e-03   1.33315771e-05   1.09114888e-04   1.25199556e-04
   2.39833807e-05]


In [246]:
print(labels)
print(trM.tolist())
np.round(trP[4:11,:])

['haze', 'primary', 'agriculture', 'clear', 'water', 'habitation', 'road', 'cultivation', 'slash_burn', 'cloudy', 'partly_cloudy', 'conventional_mine', 'bare_ground', 'artisinal_mine', 'blooming', 'selective_logging', 'blow_down']
[2.0, 0.5, 0.5, 2.0, 0.5, 0.5, 0.5, 0.5, 0.5, 2.0, 2.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]


array([[ 0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [ 0.,  1.,  1.,  1.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.]], dtype=float32)

In [247]:
rrr=pd.DataFrame(res,columns=['image_name','tags']); rrr.head(); 
suffixDT = (datetime.datetime.now()).strftime('%Y-%m-%d-%H-%M-%S'); print(suffixDT)
rrr.to_csv('../Result/vss'+suffixDT+'.csv',index=False);

2017-07-01-05-54-58
